1. Python을 통한 DB Connection

## 1-1 Postgresql 라이브러리(psycopg2)를 활용한 Connection

### 1) Connection

In [25]:
import psycopg2

# database connection
db = psycopg2.connect(
    host="127.0.0.1",
    dbname="postgres",
    user="postgres",
    password="",
    port=5432,
)

In [26]:
cursor = db.cursor()

### 2) CRUD Execution

In [7]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name VARCHAR(32), year INT, gender VARCHAR(10), count INT);"

insert_query = "INSERT INTO lecture VALUES (1, 'Tom', 2024, 'M', 1100);"

update_query = """
    UPDATE lecture
    SET id = 1,
        name = 'CHUNJAE',
        year = '1994',
        gender = 'M',
        count = '1000'
    WHERE id = 1;
    """

delete_query = "DELETE FROM lecture WHERE id = 1;"

drop_query = "DROP TABLE lecture"

In [8]:
# Create table

cursor.execute(create_query)
db.commit()

DuplicateTable: 오류:  "lecture" 이름의 릴레이션(relation)이 이미 있습니다


In [16]:
# Insert Row

cursor.execute(insert_query)
db.commit()

In [33]:
# Select

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

InFailedSqlTransaction: 오류:  현재 트랜잭션은 중지되어 있습니다. 이 트랜잭션을 종료하기 전까지는 모든 명령이 무시될 것입니다


In [38]:
# Update Row

cursor.execute(update_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

[(1, 'CHUNJAE', 1994, 'M', 1000)]

In [39]:
# DELETE ROW
cursor.execute(delete_query)
db.commit()
cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

[]

In [40]:
# DROP TABLE
cursor.execute(drop_query)
db.commit()

In [41]:
cursor.execute('select * from lecture')
cursor.fetchall()

UndefinedTable: 오류:  "lecture" 이름의 릴레이션(relation)이 없습니다
LINE 1: select * from lecture
                      ^


In [42]:
# Connection 종료

cursor.close()

### 1-2. ORM 라이브러리(sqlalchemy)를 활용한 Connection

#### ORM(Object Relational Mapping)

1) Connection 및 Session

In [55]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine_name = 'postgresql'
user_id = 'postgres'
user_pw = ''
host = '127.0.0.1'
port = 5432
database = 'postgres'

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}')

cursor = db.connect()

In [36]:
from sqlalchemy import Column, INTEGER, VARCHAR
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=db)
session = Session()

Base = declarative_base()

class lecture(Base):
    __tablename__ = 'lecture'

    id = Column('id', INTEGER, primary_key=True, autoincrement = True)

    name = Column('name', VARCHAR(10), nullable=False)

    year = Column('year', INTEGER, nullable=False)

    gender = Column('gender', VARCHAR(10), nullable=False)

    count = Column('count', INTEGER, nullable=False)

# Create
Base.metadata.create_all(db)


#### 3) ORM을 이용한 CRUD

In [37]:
# insert

datal = lecture(id=3, name='chunjae', year = 1990, gender = 'M', count=1234)

session.add(datal)
session.commit()

IntegrityError: (psycopg2.errors.UniqueViolation) 오류:  중복된 키 값이 "lecture_pkey" 고유 제약 조건을 위반함
DETAIL:  (id)=(3) 키가 이미 있습니다.

[SQL: INSERT INTO lecture (id, name, year, gender, count) VALUES (%(id)s, %(name)s, %(year)s, %(gender)s, %(count)s)]
[parameters: {'id': 3, 'name': 'chunjae', 'year': 1990, 'gender': 'M', 'count': 1234}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [38]:
# one select

res = session.query(lecture).first()

(res.id, res.name, res.year, res.gender, res.count)

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.UniqueViolation) 오류:  중복된 키 값이 "lecture_pkey" 고유 제약 조건을 위반함
DETAIL:  (id)=(3) 키가 이미 있습니다.

[SQL: INSERT INTO lecture (id, name, year, gender, count) VALUES (%(id)s, %(name)s, %(year)s, %(gender)s, %(count)s)]
[parameters: {'id': 3, 'name': 'chunjae', 'year': 1990, 'gender': 'M', 'count': 1234}]
(Background on this error at: https://sqlalche.me/e/14/gkpj) (Background on this error at: https://sqlalche.me/e/14/7s2a)

In [39]:
# update

session.query(lecture).filter(lecture.name == 'chunjae').update({'gender' : 'F','count' : 1500})

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.UniqueViolation) 오류:  중복된 키 값이 "lecture_pkey" 고유 제약 조건을 위반함
DETAIL:  (id)=(3) 키가 이미 있습니다.

[SQL: INSERT INTO lecture (id, name, year, gender, count) VALUES (%(id)s, %(name)s, %(year)s, %(gender)s, %(count)s)]
[parameters: {'id': 3, 'name': 'chunjae', 'year': 1990, 'gender': 'M', 'count': 1234}]
(Background on this error at: https://sqlalche.me/e/14/gkpj) (Background on this error at: https://sqlalche.me/e/14/7s2a)

In [40]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.orm import sessionmaker

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}')

Session = sessionmaker(db)
session = Session()

meta = MetaData()

core_table = Table(
    'first', meta,
    Column('id', Integer, primary_key = True, autoincrement = True),
    Column('name', String),
    Column('year', Integer),
    Column('gender', String),
    Column('count', Integer)
)

meta.create_all(db)


In [54]:
from sqlalchemy import insert, update, delete

# Select all
res = core_table.select()
result = session.execute(res)

for row in result:
    print(row)

In [56]:
# insert one
stmt = insert(core_table).values(name="hahaha", year=2023, gender="M", count=555)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [43]:
stmt = insert(core_table)
data_list = [
    {'name' : 'hwang', 'year' : 1994, 'gender' : 'M', 'count' : 12},
    {'name' : 'jeong', 'year' : 1996, 'gender' : 'F', 'count' : 32}
]

with db.connect() as conn:
    result = conn.execute(stmt, data_list)
    conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [44]:
stmt = update(core_table).where(core_table.c.name == 'jeong').values(year = 1000)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [53]:
stmt = delete(core_table).where(core_table.c.name =='hahaha')

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [ ]:
# drop
first_table = Table('first', Base.metadata, autoload_with=db)

first_table.drop(db)

#### 1-3. Python을 활용한  Bulk Insert

##### 1. For문을 활용한 Insert
##### 2. to_sql 함수를 활용한 Insert
##### 3. execute_values() 함수를 활용한 Insert

#### Pandas for loop

In [3]:
import psycopg2 as pgsql

# database connection
db = pgsql.connect(
    host="127.0.0.1",
    dbname="postgres",
    user="postgres",
    password="",
    port=5432,
)

cursor = db.cursor()

In [4]:
drop_query = "DROP TABLE LECTURE"

cursor.execute(drop_query)
db.commit()

In [61]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name varchar(65533), year INT, gender varchar(65533), count INT);"

cursor.execute(create_query)
db.commit()

In [18]:
import pandas as pd

df = pd.read_csv('./dataset/data-01/names.csv')
df.head(10)

,id,name,year,gender,count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746
5,6,Margaret,1880,F,1578
6,7,Ida,1880,F,1472
7,8,Alice,1880,F,1414
8,9,Bertha,1880,F,1320
9,10,Sarah,1880,F,1288


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   int64 
 1   name    2000 non-null   object
 2   year    2000 non-null   int64 
 3   gender  2000 non-null   object
 4   count   2000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 78.3+ KB


In [ ]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite://{}')



# Bulk Insert

df.to_sql(name='lecture', con=db, if_exists = 'replace')

In [21]:
# Bulk insert -> tuple(for)

for data in range(len(df)):
    datas = tuple(df.iloc[data])
    query = f'insert into lecture values {datas}'
    cursor.execute(query)

db.commit()

In [24]:
# select
cursor.execute('select * from lecture limit 10')
cursor.fetchall()

UndefinedTable: 오류:  "lecture" 이름의 릴레이션(relation)이 없습니다
LINE 1: select * from lecture limit 10
                      ^


#### Pandas - to_sql 메소드
    - dataframe.to_sql(테이블명, sqlalchemy_connector, option)

In [5]:
from sqlalchemy import create_engine

engine = 'postgresql'
user = 'postgres'
password = ''
host = '127.0.0.1'
port = '5432'
database = 'postgres'

db = create_engine(f'{engine}://{user}:{password}@{host}:{port}/{database}')



In [6]:
import pandas as pd

df = pd.read_csv('./dataset/data-01/names.csv')

df.iloc[1]
df.iloc[1,:]

id           2
name      Anna
year      1880
gender       F
count     2604
Name: 1, dtype: object

In [7]:
# bulk insert

df.to_sql(name = 'lecture', con = db, if_exists= 'replace')

1000

### 3) psycopg2 라이브러리의 execute_values() 메소드

In [54]:
import psycopg2 as pgsql

# database connection
db = pgsql.connect(
    host="127.0.0.1",
    dbname="postgres",
    user="postgres",
    password="",
    port=5432,
)

cursor = db.cursor()

In [51]:
delete_query = 'delete from lecture'

cursor.execute(delete_query)
db.commit()

In [29]:
cursor.execute('select * from lecture')
cursor.fetchall()

[]

In [45]:
import pandas as pd

df = pd.read_csv('./dataset/data-01/names.csv')
df['id'] = df['id'].astype(int)
df['year'] = df['year'].astype(int)
df['count'] = df['count'].astype(int)

insert_list = list()

for i in range(5,50):
    insert_list.append(tuple(df.iloc[i]))

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   int32 
 1   name    2000 non-null   object
 2   year    2000 non-null   int32 
 3   gender  2000 non-null   object
 4   count   2000 non-null   int32 
dtypes: int32(3), object(2)
memory usage: 54.8+ KB


In [58]:
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)


In [59]:
from psycopg2.extras import execute_values

sql = f'INSERT INTO lecture VALUES %s;'
execute_values(cursor, sql, insert_list)

db.commit()

ProgrammingError: can't adapt type 'numpy.int32'

### 2. dotenv 라이브러리를 활용한 민감 정보 관리
- 환경 변수에 대한 관리를 효과적이고 안전하게 할 수 있도록 도와주는 python 라이브러리
- DB정보, 비밀번호, API KEY 등 외부에 공유되거나 git에 오라가면 안되는 값들을 하드코딩 하지 않고 사용

pip install python-dotenv

In [12]:
import dotenv

env_path = dotenv.find_dotenv()
print(env_path)

d:\workspace\Chunjae_Bigdata_9th\10.25~10.31\.env


In [13]:
# .env load
dotenv.load_dotenv(env_path)

True

In [21]:
# .env values

# dotenv.dotenv_values(env_path)

In [70]:
# .env values change
# dotenv.set_key(env_path, 'HOST', '87654321')

(True, 'HOST', '87654321')

In [20]:
# load

import os

dotenv.load_dotenv()

engine = os.getenv('POSTGRES_ENGINE')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")

# print(engine, user, password, host, port, database)

### DB Connection

In [19]:
import psycopg2

db = psycopg2.connect(
    dbname = database,
    user = user,
    password = password,
    port = port,
    host = host
)

cursor = db.cursor()

### 3. Python Class를 사용한 DB Connection

0) self

In [24]:
class TempClass:
    def __init__(self, name:str, age:int, count:int):
        self.name = name
        self.age = age
        self.count = count
        self.addage()

    def printall(self):
        print(self.name)
        print(self.age)
        print(self.count)


    def printself(self):
        return self
    
    def addage(self):
        self.new_age = self.age + 10


In [26]:
a1 = TempClass('chunjae', 2,3)
s1 = a1.printself()

print(a1)
print(s1)

In [27]:
# 객체 인자 확인
s1.__dict__

{'name': 'chunjae', 'age': 2, 'count': 3, 'new_age': 12}

In [28]:
[i for i in dir(s1) if '__' not in i]

['addage', 'age', 'count', 'name', 'new_age', 'printall', 'printself']

In [30]:
from easydict import EasyDict

self = EasyDict(dict())
self['name'] = 'kwak'
self['age'] = 3
self['count'] = 1

s2 = self

s2.age

3

In [32]:
self.hoho = 'ho'

self

{'name': 'kwak', 'age': 3, 'count': 1, 'hoho': 'ho'}

### 1) DBConnector Class

In [16]:
import psycopg2

class DBConnector:
    def __init__(self, host, database, user, password, port):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port

    def postgres_connect(self):
        print('<<< pssql_connector >>>')
        self.conn = psycopg2.connect(
            host=self.host,
            dbname = self.database,
            user = self.user,
            password = self.password,
            port = self.port,
        )
        return self

In [ ]:
host, database, user, password, port

In [10]:
db_object = DBConnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor=db_conn.cursor()

<<< pssql_connector >>>


In [11]:
cursor.execute("SELECT * FROM lecture LIMIT 5")

cursor.fetchall()

[(0, 1, 'Mary', 1880, 'F', 7065),
 (1, 2, 'Anna', 1880, 'F', 2604),
 (2, 3, 'Emma', 1880, 'F', 2003),
 (3, 4, 'Elizabeth', 1880, 'F', 1939),
 (4, 5, 'Minnie', 1880, 'F', 1746)]

with문 :  python에서 파일 또는 리소스 관리를 더 효과적으로 처리하기 위한 블록 구조
리소스를 열고 사용한 후 자동으로 닫아주기 때문에 닫아주는 코드를 작성할 필요가 없다.

In [12]:
file = open('temp.txt', 'a')
file.write('1')
file.write('2')

file.close()

In [15]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

TypeError: 'DBConnector' object does not support the context manager protocol

In [19]:
class ContextManager:
    def __init__(self, age):
        print('Start! Class')
        self.age = age

    def __enter__(self):
        print('Enter')

    def __exit__(self, exc_type, exc_value, traceback):
        print('Exit')

    def show(self):
        print(self.age)
        return self.__dict__

In [20]:
context = ContextManager(age=10)

with context:
    print(context.show())

Start! Class
Enter
10
{'age': 10}
Exit


In [31]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        print('start class')
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.connect = self.postgres_connect()

    def __enter__(self):
        print('Enter')
        return self
    
    def __exit__(self,exc_type, exc_value, traceback):
        self.conn.close()
        print('exit')
    
    def postgres_connect(self):
        self.conn = psycopg2.connect(
            host = self.host,
            dbname = self.database,
            user = self.user,
            password = self.password,
            port = self.port
        )
        return self

In [32]:
db_object = DBConnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor = db_conn.cursor()

start class


In [34]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor = db_conn.cursor()

    cursor.execute('select * from lecture limit 10')
    print(cursor.fetchall())

start class
Enter
[(0, 1, 'Mary', 1880, 'F', 7065), (1, 2, 'Anna', 1880, 'F', 2604), (2, 3, 'Emma', 1880, 'F', 2003), (3, 4, 'Elizabeth', 1880, 'F', 1939), (4, 5, 'Minnie', 1880, 'F', 1746), (5, 6, 'Margaret', 1880, 'F', 1578), (6, 7, 'Ida', 1880, 'F', 1472), (7, 8, 'Alice', 1880, 'F', 1414), (8, 9, 'Bertha', 1880, 'F', 1320), (9, 10, 'Sarah', 1880, 'F', 1288)]
exit


In [52]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port = port
        )
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port

    def __enter__(self):
        print("enter")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("Exit")

    def postgres_connect(self):
        print("<<< postgres_connect >>>")
        self.conn = psycopg2.connect(
            host = self.host,
            dbname = self.database,
            user = self.user,
            password = self.password,
            port = self.port,
            options='-c client_encoding=UTF8'
        )
        return self
    


#### Asterisk(*)

In [38]:
def func(one, two):
    print('one = ', one)
    print('two = ', two)
    
func(1,2)

one =  1
two =  2


In [44]:
# *args = 이름이 정해지지 않은 변수가 여러개 있을 때 사용 가능 (튜플로 감싸져 나옴)

def func(*args):
    print('args = ', args)
    print(sum(args))

    
func(1,2,3,4,5)

args =  (1, 2, 3, 4, 5)
15


In [45]:
# **kwargs = 더블 애스터리스크 : 이름이 정해진 변수가 여러개 있을 때 사용

def func(**kwargs):
    print("**kwargs = ", kwargs)
    
func(one=1, two=4)

**kwargs =  {'one': 1, 'two': 4}


In [53]:
test = DBConnector(1,2,3,4,5)

In [54]:
# list

args = [1,2,3,4,5]

list_params = DBConnector(*args)

list_params.conn_params

{'host': 1, 'dbname': 2, 'user': 3, 'password': 4, 'port': 5}

In [55]:
# dict

kwargs = dict(
    host = 1,
    database = 2,
    user = 3,
    password = 4,
    port = 5
)

dict_params = DBConnector(**kwargs)

dict_params.conn_params

{'host': 1, 'dbname': 2, 'user': 3, 'password': 4, 'port': 5}

In [56]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        print('start DBConnector!')
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port = port
        )
        self.connect = self.postgres_connect()

    def __enter__(self):
        print('enter')
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print('exit')

    def postgres_connect(self):
        self.conn = psycopg2.connect(
            **self.conn_params
        )
        return self

In [59]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor = db_conn.cursor()

    cursor.execute('select * from lecture limit 5')
    print(cursor.fetchall())

start DBConnector!
enter
[(0, 1, 'Mary', 1880, 'F', 7065), (1, 2, 'Anna', 1880, 'F', 2604), (2, 3, 'Emma', 1880, 'F', 2003), (3, 4, 'Elizabeth', 1880, 'F', 1939), (4, 5, 'Minnie', 1880, 'F', 1746)]
exit


#### .env 적용한 connection

In [62]:
import dotenv, psycopg2

env_path = dotenv.find_dotenv()
env_path

dotenv.load_dotenv(env_path)

import os

engine = os.getenv('POSTGRES_ENGINE')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")



In [63]:
db_settings = {
    "postgres" : {
        'host' : os.getenv('POSTGRES_HOST'),
        'database' :  os.getenv("POSTGRES_DB"),
        'user' : os.getenv('POSTGRES_USER'),
        'password' : os.getenv('POSTGRES_PASSWORD'),
        'port' : os.getenv("POSTGRES_PORT")
    }
}

In [ ]:
db_settings['postgres']

In [69]:
db_object = DBConnector(**db_settings['postgres'])

with db_object as connected:
    db_conn = connected.conn
    cursor = db_conn.cursor()

    cursor.execute('select * from lecture limit 5')
    print(cursor.fetchall())

start DBConnector!
enter
[(0, 1, 'Mary', 1880, 'F', 7065), (1, 2, 'Anna', 1880, 'F', 2604), (2, 3, 'Emma', 1880, 'F', 2003), (3, 4, 'Elizabeth', 1880, 'F', 1939), (4, 5, 'Minnie', 1880, 'F', 1746)]
exit


In [70]:
db_object = DBConnector(**db_settings['postgres'])

with db_object as connected:
    db_conn = connected.conn
    cursor = db_conn.cursor()

    cursor.execute('select table_name from information_schema.tables')
    print(cursor.fetchall())

start DBConnector!
enter
[('pg_statistic',), ('pg_type',), ('pg_foreign_table',), ('pg_authid',), ('pg_shadow',), ('pg_roles',), ('pg_statistic_ext_data',), ('pg_hba_file_rules',), ('pg_settings',), ('pg_file_settings',), ('pg_backend_memory_contexts',), ('pg_ident_file_mappings',), ('pg_config',), ('pg_shmem_allocations',), ('pg_tables',), ('lecture',), ('pg_user_mapping',), ('pg_statio_all_sequences',), ('pg_replication_origin_status',), ('pg_subscription',), ('pg_attribute',), ('pg_proc',), ('pg_class',), ('pg_attrdef',), ('pg_statio_sys_sequences',), ('pg_statio_user_sequences',), ('pg_constraint',), ('pg_inherits',), ('pg_index',), ('pg_operator',), ('pg_opfamily',), ('pg_opclass',), ('pg_am',), ('pg_amop',), ('pg_amproc',), ('pg_language',), ('pg_largeobject_metadata',), ('pg_aggregate',), ('pg_statistic_ext',), ('pg_rewrite',), ('pg_trigger',), ('pg_event_trigger',), ('pg_description',), ('pg_cast',), ('pg_enum',), ('pg_namespace',), ('pg_conversion',), ('pg_depend',), ('pg_data